In [7]:
from qdrant_client import QdrantClient
# from qdrant_client.models import SparseVectorParams
from qdrant_client.http import models
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http.models import PointStruct
import numpy as np
import random
import pandas as pd
import time

import utils

In [1]:
import utils
from qdrant_client import QdrantClient
from qdrant_client.http import models

In [2]:
qdrantClient = QdrantClient(host='localhost', port=6333, timeout=10000000)
# base_vectors = utils.read_fvecs("../../dataset/siftsmall/siftsmall_base.fvecs")
# query_vectors = utils.read_fvecs("../../dataset/siftsmall/siftsmall_query.fvecs")
# knn_groundtruth = utils.read_ivecs("../../dataset/siftsmall/siftsmall_groundtruth.ivecs")
base_vectors = utils.read_fvecs("../../dataset/sift/sift_base.fvecs")
query_vectors = utils.read_fvecs("../../dataset/sift/sift_query.fvecs")
knn_groundtruth = utils.read_ivecs("../../dataset/sift/sift_groundtruth.ivecs")

Loading file: sift_base.fvecs
    The dimension of the vectors in the file is: 128
    The final shape of the loaded dataset sift_base.fvecs is (1000000, 128).
Loading file: sift_query.fvecs
    The dimension of the vectors in the file is: 128
    The final shape of the loaded dataset sift_query.fvecs is (10000, 128).
 Loading file: sift_groundtruth.ivecs
    The dimension of the vectors in the file is: 100
    The final shape of the loaded dataset is (10000, 100).


In [3]:
collection_name = "test_multivector"
qdrantClient.delete_collection(collection_name=collection_name)

qdrantClient.recreate_collection(
    collection_name=collection_name,
    vectors_config={
        "text-dense": models.VectorParams(
            size=1536,  # OpenAI Embeddings
            distance=models.Distance.COSINE,
        )
    },
    sparse_vectors_config={
        "text-sparse": models.SparseVectorParams(
            index=models.SparseIndexParams(
                on_disk=False,
            )
        )
    },
)

True

In [ ]:
query_indices, query_values = compute_sparse_vector(query_text)
query_dense_vector = compute_dense_vector(query_text)


In [4]:
vector_size = 128
collection_name = "test_multivector"

In [35]:
base_vectors = base_vectors[:50000]

In [36]:
len(base_vectors)

50000

In [37]:
qdrantClient.upsert(
    collection_name=collection_name,
    points = models.Batch(
    ids=list(range(len(base_vectors))),  # ids from 0 to len(base_vectors) - 1
    payloads=[{ "attr1": random.choice([True, False]),
                "attr2": random.choice([True, False]),
                "attr3": random.choice([True, False])}
                for _ in range(len(base_vectors))],
    vectors=base_vectors.tolist()  # Convert base_vectors to a list
    )
)



UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [9]:
batch_points = [PointStruct(id=i, vector=elem, payload= {"attr1": random.choice([True, False]), "attr2": random.choice([True, False]), "attr3": random.choice([True, False])}) for i, elem in enumerate(base_vectors)]


In [13]:
batch_points = [PointStruct(id=i, vector=elem["vector"], payload= {"attr1": elem["attr1"], "attr2": elem["attr2"], "attr3": elem["attr3"]}) for i, elem in base_vectors_with_attributes.iterrows()]
print(batch_points)

In [10]:

operation_info = qdrantClient.upsert(
    collection_name=collection_name,
    wait=True,
    points=batch_points
)
print(operation_info)

UnexpectedResponse: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Payload error: JSON payload (664318629 bytes) is larger than allowed (limit: 33554432 bytes)."},"time":0.0}'

In [7]:
qdrantClient.get_collection(collection_name=collection_name)


CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=0, indexed_vectors_count=0, points_count=0, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=128, distance=<Distance.EUCLID: 'Euclid'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

In [7]:
query_vectors_with_attributes = pd.DataFrame({'vector': query_vectors.tolist()})
num_rows = len(query_vectors_with_attributes)
query_vectors_with_attributes['attr1'] = [random.choice([True, False]) for _ in range(num_rows)]
query_vectors_with_attributes['attr2'] = [random.choice([True, False]) for _ in range(num_rows)]
query_vectors_with_attributes['attr3'] = [random.choice([True, False]) for _ in range(num_rows)]

In [9]:
search_result = qdrantClient.search(
        collection_name=collection_name, 
        # query_vector=vec, 
        query_filter=models.Filter(
            # must = AND
            must=[
                models.FieldCondition(
                    key="attr1",
                    match=models.MatchValue(
                        value=True,
                    ),
                )
            ]
        ),
)

TypeError: search() missing 1 required positional argument: 'query_vector'